# Atomap tutorial: finding and analysing sublattices

For more details see the open access article: **Atomap: a new software tool for the automated analysis of atomic resolution images using two-dimensional Gaussian fitting**. https://dx.doi.org/10.1186/s40679-017-0042-5

This tutorial shows how to use Atomap to analyse atomic structures, with a focus on perovskite oxides.

More documentation is found at https://atomap.org/

## Importing the libraries

Firstly, we must set the plotting toolkit:

Atomap relies heavily on HyperSpy for visualization and fitting, and uses HyperSpy signals for most of the outputs. So we need to import both HyperSpy and Atomap.

You might get a "WARNING:hyperspy_gui_traitsui", this can be ignored.

### Loading data

Atomap uses HyperSpy signals as its input, which can be any loaded from many different types of files. DM3/DM4, tif, emi/ser, jpg or HDF5-files. This can be loaded using `s = hs.load(your_filename)`.

Here we will be using a test dataset, generated by the `dummy_data` module: `get_fantasite`

This is an imaginary structure, similar to a perovskite oxide projected along on the [110] direction. We visualize it using the `plot` method.

## Finding initial positions

Our first task is finding and fitting the most intense atomic columns. This is done by using a peak finder, where the minimum distance between the features is used as an input parameter. This function returns a HyperSpy signal, where the atom positions are saved as permanent markers in the metadata

In this plot, the minimum feature separation is shown on the x-axis for the navigation plot. This parameter can be changed by using the arrow keys. For this image, we want to get the peaks for only the most intense atomic columns.
A feature separation of 12 works fine.

We use this value as an input for the next step, which involves getting these atomic positions as a list.

To add or remove atoms, use `am.add_atoms_with_gui`, see https://atomap.org/gui_functions.html#adding-atoms-using-gui for more info.

Also, atoms from different regions can be selected interactively using `am.select_atoms_with_gui`, see https://atomap.org/gui_functions.html#selecting-atoms-with-gui

## Making a sublattice

The atom positions are used to initialize the sublattice object.

This sublattice object contains the atom positions we just found, the image, and many utility functions. For example a plot function.

The first step is to refine the atomic positions using center of mass, since the initial atom positions are not very good.

Then using 2D Gaussians

This will give much more accurate atomic positions:

## Constructing zone axes

The next step is to find the relation between the atom positions, by ordering them into atomic planes:

These atomic planes can be visualized. Navigate using the arrow keys. The number to the top left is the direction of the zone axis.

## Finding the second sublattice

In this structure, we have two separate sublattices. Having found the first one, we can use those atomic positions to find the second one. Here we use the fact that the atoms in the second sublattice is located between the atoms in the second zone axis for the first sublattice.

Then we pass this to the `find_missing_atoms_from_zone_vector`, which returns a new list of positions

Secondly, we "subtract" the intensity of the `sublattice` atoms from the fantasite image

Which we use to make the second sublattice:

And refine the positions

## The atom lattice object

Having made these two sublattices, we can combine them in an `Atom_Lattice` object

Atom_Lattice objects can be stored, to avoid having to run the often slow fitting routines several times.

Which can be restored:

## Analysing the structure

The sublattice object contains many utility functions for visualizing various structural effects.

For example, the distance between the atoms.

Or the variations in distance between the atoms:

Or the ellipticity of the atoms, which can be used to extract information about the structure parallel to the electron beam. Plotting only the magnitude:

Plotting both the magnitude, and the direction:

## Integrating intensity

The intensity of the atomic columns can be found using `integrate_column_intensity` function, which supports two different methods: `Watershed` and `Voronoi`, where the latter is the default one.

This also works for sublattices directly. Since the fantasite dataset contain several sublattices, the results here will contain intensity from several columns. So for situation like this, the `atom_lattice.integrate_column_intensity` should be used.

## Line profiles

All these structural properties can also be plotted as line profiles, as a function of distance from some atomic plane. For example the distance between the atoms in the horizontal direction.

This requires two things: 1) the zone axis perpendicular to the horizontal direction, and 2) some atomic plane perpendicular to the horizontal direction.

The former is found using `plot_planes`

Which shows that the second zone axis is the one perpendicular to the horisontal axis. Which we get from `zones_axis_average_distances`. Note that Python is zero-indexed, meaning we need to get index `1`

Then we extract an atomic plane which is also perpendicular to the horizontal axis

Lastly, we get the line profile:

For all the `get_..._line_profile` methods, the raw non-interpolated data can be accessed in the signal:

This can be done for other properties as well:

## Properties

The atomic positions can be accessed directly as well

These properites include: `y_position`, `sigma_x`, `sigma_y`, `ellipticity`, `rotation_ellipticity`.

These can be visualized using a variety of methods in the `sublattice` object, or stored to (for example) a csv-file which can be opened in spreadsheet software

The easiest way to save the monolayer map from earlier is through the HyperSpy HDF5 format

These signals can also be saved as tif, so they can be opened in other image viewers. For this to work, we must first change NaN values to zero:

This file can be opened in image viewers such as ImageJ or Digital Micrograph

## Determining polarization

In many ferroelectric materials, the spontaneous electric polarization can be determined by looking at the shift of some atomic columns in relation to the others. One example of this is in the ferroelectric perovskite oxides, where the B-cation is shifted from its cubic centrosymmetric position. The polarization can be determined by finding both the direction and magnitude of this shift.

Firstly, we get an appropriate artificial dataset, resembling a ferroelectric thin film grown on top of a non-ferroelectric substrate.

The blue, B-cation, atom columns in the top part of the image are shifted towards the left. By finding the centre position of four neighboring red A-cation forming a square, this shift can be quantified.

Finding these neighbors relies on moving along two zone axis directions in the A-cation sublattice.

Next, find the two perpendicular zone axes spanning this square. For the perovskite oxide 100 projection, this is most likely the two first ones.

The zone axes are then used with the B-cation sublattice:

This can be visualized directly by using the ``plot`` method, and the data itself can be accessed in the signal’s metadata.

## Calculating pair distribution function

A method of analysis common in X-ray diffraction is the plotting of a pair distribution function (PDF). This function calculates the distance between every atom with every other atom in the sampled volume. This function can also be performed in two dimensions from an image of atoms. A PDF of a crystalline material is similar to a radially averaged diffraction pattern. They are particularly useful for looking for ordering in amorphous materials.

This returns a HyperSpy signal, which can be plotted using the standard plotting function.